In [66]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
#import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [67]:
import keras.applications
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

In [96]:
model_weights_checkpoint = 'deep_transfer_3_class_v3_ck.h5'

In [97]:
import keras
#from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(model_weights_checkpoint, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [99]:
img_width, img_height = 299, 299

In [100]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(img_width, img_height, 3))  # this assumes K.image_data_format() == 'channels_last'

In [102]:
base_model = keras.applications.InceptionResNetV2(weights='imagenet', include_top=False)
#base_model = keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
#base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [103]:
# add a global spatial average pooling layer
x = base_model.output

In [104]:
# let's add a fully-connected layer
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

In [105]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [106]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [107]:
# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='sgd', loss='categorical_crossentropy')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [108]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_148 (Activation)     (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
mixed_5b (Concatenate)          (None, None, None, 3 0           activation_153[0][0]             
                                                                 activation_155[0][0]             
                                                                 activation_158[0][0]             
                                                                 activation_159[0][0]             
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, None, None, 3 10240       mixed_5b[0][0]                   
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, None, None, 3 96          conv2d_16[0][0]                  
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
activation_166 (Activation)     (None, None, None, 3 0           batch_normalization_19[0][0]     
__________________________________________________________________________________________________
activation_168 (Activation)     (None, None, None, 3 0           batch_normalization_21[0][0]     
__________________________________________________________________________________________________
activation_171 (Activation)     (None, None, None, 6 0           batch_normalization_24[0][0]     
__________________________________________________________________________________________________
block35_2_mixed (Concatenate)   (None, None, None, 1 0           activation_166[0][0]             
                                                                 activation_168[0][0]             
                                                                 activation_171[0][0]             
__________

__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, None, None, 3 10240       block35_3_ac[0][0]               
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, None, None, 3 9216        activation_179[0][0]             
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, None, None, 6 27648       activation_182[0][0]             
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, None, None, 3 96          conv2d_31[0][0]                  
__________________________________________________________________________________________________
batch_normalization_33 (BatchNo (None, None, None, 3 96          conv2d_33[0][0]                  
__________

__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, None, None, 3 10240       block35_5_ac[0][0]               
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, None, None, 4 13824       activation_193[0][0]             
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, None, None, 3 96          conv2d_44[0][0]                  
__________________________________________________________________________________________________
batch_normalization_47 (BatchNo (None, None, None, 4 144         conv2d_47[0][0]                  
__________________________________________________________________________________________________
activation_191 (Activation)     (None, None, None, 3 0           batch_normalization_44[0][0]     
__________

...

mixed_7a (Concatenate)          (None, None, None, 2 0           activation_305[0][0]             
                                                                 activation_307[0][0]             
                                                                 activation_310[0][0]             
                                                                 max_pooling2d_7[0][0]            
__________________________________________________________________________________________________
conv2d_165 (Conv2D)             (None, None, None, 1 399360      mixed_7a[0][0]                   
__________________________________________________________________________________________________
batch_normalization_165 (BatchN (None, None, None, 1 576         conv2d_165[0][0]                 
__________________________________________________________________________________________________
activation_312 (Activation)     (None, None, None, 1 0           batch_normalization_165[0][0]    
__________

__________________________________________________________________________________________________
activation_321 (Activation)     (None, None, None, 2 0           batch_normalization_174[0][0]    
__________________________________________________________________________________________________
conv2d_172 (Conv2D)             (None, None, None, 1 399360      block8_2_ac[0][0]                
__________________________________________________________________________________________________
conv2d_175 (Conv2D)             (None, None, None, 2 172032      activation_321[0][0]             
__________________________________________________________________________________________________
batch_normalization_172 (BatchN (None, None, None, 1 576         conv2d_172[0][0]                 
__________________________________________________________________________________________________
batch_normalization_175 (BatchN (None, None, None, 2 768         conv2d_175[0][0]                 
__________

__________________________________________________________________________________________________
block8_5_mixed (Concatenate)    (None, None, None, 4 0           activation_327[0][0]             
                                                                 activation_330[0][0]             
__________________________________________________________________________________________________
block8_5_conv (Conv2D)          (None, None, None, 2 933920      block8_5_mixed[0][0]             
__________________________________________________________________________________________________
block8_5 (Lambda)               (None, None, None, 2 0           block8_4_ac[0][0]                
                                                                 block8_5_conv[0][0]              
__________________________________________________________________________________________________
block8_5_ac (Activation)        (None, None, None, 2 0           block8_5[0][0]                   
__________

__________________________________________________________________________________________________
activation_340 (Activation)     (None, None, None, 1 0           batch_normalization_193[0][0]    
__________________________________________________________________________________________________
conv2d_194 (Conv2D)             (None, None, None, 2 129024      activation_340[0][0]             
__________________________________________________________________________________________________
batch_normalization_194 (BatchN (None, None, None, 2 672         conv2d_194[0][0]                 
__________________________________________________________________________________________________
activation_341 (Activation)     (None, None, None, 2 0           batch_normalization_194[0][0]    
__________________________________________________________________________________________________
conv2d_192 (Conv2D)             (None, None, None, 1 399360      block8_7_ac[0][0]                
__________

__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, None, None, 2 768         conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_347 (Activation)     (None, None, None, 1 0           batch_normalization_200[0][0]    
__________________________________________________________________________________________________
activation_350 (Activation)     (None, None, None, 2 0           batch_normalization_203[0][0]    
__________________________________________________________________________________________________
block8_10_mixed (Concatenate)   (None, None, None, 4 0           activation_347[0][0]             
                                                                 activation_350[0][0]             
__________________________________________________________________________________________________
block8_10_

In [109]:
from keras.preprocessing.image import ImageDataGenerator

In [110]:
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb-subset\\validation'
nb_train_samples = 2500
nb_validation_samples = 560
epochs = 100
batch_size = 32
samples_per_epoch= 5000

In [111]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #horizontal_flip=True
    width_shift_range=0.2,
    height_shift_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [112]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2500 images belonging to 3 classes.
Found 560 images belonging to 3 classes.


In [113]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

# fine-tune the model
model.fit_generator(
    train_generator,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpoint, history ])

Epoch 1/100
77/78 [============================>.] - ETA: 20:09 - loss: 0.6781 - acc: 0.65 - ETA: 12:59 - loss: 0.6706 - acc: 0.63 - ETA: 10:33 - loss: 0.6410 - acc: 0.65 - ETA: 9:17 - loss: 0.6574 - acc: 0.6406 - ETA: 8:30 - loss: 0.6701 - acc: 0.643 - ETA: 7:57 - loss: 0.6810 - acc: 0.633 - ETA: 7:32 - loss: 0.6791 - acc: 0.635 - ETA: 7:12 - loss: 0.6741 - acc: 0.638 - ETA: 6:55 - loss: 0.6750 - acc: 0.636 - ETA: 6:41 - loss: 0.6797 - acc: 0.635 - ETA: 6:28 - loss: 0.6835 - acc: 0.635 - ETA: 6:17 - loss: 0.6829 - acc: 0.635 - ETA: 6:07 - loss: 0.6759 - acc: 0.641 - ETA: 5:57 - loss: 0.6755 - acc: 0.641 - ETA: 5:49 - loss: 0.6805 - acc: 0.636 - ETA: 5:40 - loss: 0.6812 - acc: 0.635 - ETA: 5:32 - loss: 0.6813 - acc: 0.637 - ETA: 5:25 - loss: 0.6790 - acc: 0.639 - ETA: 5:18 - loss: 0.6779 - acc: 0.640 - ETA: 5:11 - loss: 0.6743 - acc: 0.641 - ETA: 5:04 - loss: 0.6735 - acc: 0.642 - ETA: 4:57 - loss: 0.6738 - acc: 0.641 - ETA: 4:51 - loss: 0.6736 - acc: 0.642 - ETA: 4:44 - loss: 0.6737 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.6503 - acc: 0.635 - ETA: 6:06 - loss: 0.6284 - acc: 0.661 - ETA: 6:01 - loss: 0.6394 - acc: 0.652 - ETA: 5:57 - loss: 0.6469 - acc: 0.653 - ETA: 5:52 - loss: 0.6399 - acc: 0.658 - ETA: 5:47 - loss: 0.6438 - acc: 0.663 - ETA: 5:42 - loss: 0.6517 - acc: 0.657 - ETA: 5:37 - loss: 0.6495 - acc: 0.658 - ETA: 5:32 - loss: 0.6461 - acc: 0.660 - ETA: 5:28 - loss: 0.6448 - acc: 0.660 - ETA: 5:23 - loss: 0.6426 - acc: 0.662 - ETA: 5:18 - loss: 0.6430 - acc: 0.661 - ETA: 5:13 - loss: 0.6428 - acc: 0.659 - ETA: 5:08 - loss: 0.6404 - acc: 0.660 - ETA: 5:04 - loss: 0.6408 - acc: 0.659 - ETA: 4:59 - loss: 0.6422 - acc: 0.660 - ETA: 4:54 - loss: 0.6405 - acc: 0.661 - ETA: 4:49 - loss: 0.6423 - acc: 0.659 - ETA: 4:44 - loss: 0.6423 - acc: 0.660 - ETA: 4:39 - loss: 0.6406 - acc: 0.658 - ETA: 4:23 - loss: 0.6461 - acc: 0.659 - ETA: 4:19 - loss: 0.6479 - acc: 0.659 - ETA: 4:15 - loss: 0.6485 - acc: 0.658 - ETA: 4:11 - loss: 0.6481 - acc: 0.657 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.6342 - acc: 0.666 - ETA: 6:05 - loss: 0.6410 - acc: 0.656 - ETA: 6:01 - loss: 0.6380 - acc: 0.659 - ETA: 5:56 - loss: 0.6356 - acc: 0.671 - ETA: 5:51 - loss: 0.6345 - acc: 0.677 - ETA: 5:46 - loss: 0.6252 - acc: 0.687 - ETA: 5:42 - loss: 0.6224 - acc: 0.686 - ETA: 5:37 - loss: 0.6237 - acc: 0.684 - ETA: 5:32 - loss: 0.6234 - acc: 0.684 - ETA: 5:27 - loss: 0.6200 - acc: 0.684 - ETA: 5:22 - loss: 0.6192 - acc: 0.685 - ETA: 5:18 - loss: 0.6157 - acc: 0.685 - ETA: 5:13 - loss: 0.6148 - acc: 0.684 - ETA: 5:08 - loss: 0.6134 - acc: 0.685 - ETA: 5:03 - loss: 0.6135 - acc: 0.686 - ETA: 4:58 - loss: 0.6172 - acc: 0.682 - ETA: 4:53 - loss: 0.6205 - acc: 0.679 - ETA: 4:49 - loss: 0.6199 - acc: 0.679 - ETA: 4:44 - loss: 0.6196 - acc: 0.677 - ETA: 4:39 - loss: 0.6191 - acc: 0.676 - ETA: 4:34 - loss: 0.6204 - acc: 0.674 - ETA: 4:29 - loss: 0.6203 - acc: 0.673 - ETA: 4:25 - loss: 0.6204 - acc: 0.672 - ETA: 4:20 - loss: 0.6201 - acc: 0.672 -

77/78 [============================>.] - ETA: 6:12 - loss: 0.6103 - acc: 0.666 - ETA: 6:06 - loss: 0.5948 - acc: 0.677 - ETA: 6:01 - loss: 0.5984 - acc: 0.677 - ETA: 5:56 - loss: 0.5985 - acc: 0.682 - ETA: 5:51 - loss: 0.6034 - acc: 0.677 - ETA: 5:46 - loss: 0.6036 - acc: 0.678 - ETA: 5:42 - loss: 0.6027 - acc: 0.677 - ETA: 5:37 - loss: 0.6059 - acc: 0.677 - ETA: 5:32 - loss: 0.6096 - acc: 0.677 - ETA: 5:27 - loss: 0.6086 - acc: 0.679 - ETA: 5:22 - loss: 0.6038 - acc: 0.682 - ETA: 5:18 - loss: 0.6008 - acc: 0.684 - ETA: 5:13 - loss: 0.5994 - acc: 0.685 - ETA: 5:08 - loss: 0.6007 - acc: 0.684 - ETA: 5:03 - loss: 0.6019 - acc: 0.684 - ETA: 4:58 - loss: 0.6024 - acc: 0.685 - ETA: 4:54 - loss: 0.6022 - acc: 0.686 - ETA: 4:49 - loss: 0.6002 - acc: 0.688 - ETA: 4:44 - loss: 0.5997 - acc: 0.688 - ETA: 4:39 - loss: 0.5996 - acc: 0.687 - ETA: 4:34 - loss: 0.5999 - acc: 0.687 - ETA: 4:29 - loss: 0.5977 - acc: 0.688 - ETA: 4:25 - loss: 0.5982 - acc: 0.688 - ETA: 4:20 - loss: 0.6004 - acc: 0.686 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.6019 - acc: 0.677 - ETA: 6:06 - loss: 0.5863 - acc: 0.697 - ETA: 6:01 - loss: 0.5820 - acc: 0.708 - ETA: 5:56 - loss: 0.5760 - acc: 0.705 - ETA: 5:51 - loss: 0.5835 - acc: 0.693 - ETA: 5:46 - loss: 0.5798 - acc: 0.699 - ETA: 5:41 - loss: 0.5778 - acc: 0.696 - ETA: 5:37 - loss: 0.5793 - acc: 0.691 - ETA: 5:32 - loss: 0.5805 - acc: 0.688 - ETA: 5:27 - loss: 0.5765 - acc: 0.690 - ETA: 4:57 - loss: 0.5841 - acc: 0.688 - ETA: 4:55 - loss: 0.5815 - acc: 0.688 - ETA: 4:52 - loss: 0.5809 - acc: 0.687 - ETA: 4:49 - loss: 0.5800 - acc: 0.686 - ETA: 4:46 - loss: 0.5807 - acc: 0.684 - ETA: 4:42 - loss: 0.5806 - acc: 0.684 - ETA: 4:39 - loss: 0.5829 - acc: 0.685 - ETA: 4:35 - loss: 0.5818 - acc: 0.685 - ETA: 4:31 - loss: 0.5791 - acc: 0.688 - ETA: 4:27 - loss: 0.5802 - acc: 0.690 - ETA: 4:23 - loss: 0.5808 - acc: 0.690 - ETA: 4:19 - loss: 0.5828 - acc: 0.689 - ETA: 4:15 - loss: 0.5813 - acc: 0.692 - ETA: 4:11 - loss: 0.5819 - acc: 0.692 -

...

77/78 [============================>.] - ETA: 6:10 - loss: 0.3387 - acc: 0.864 - ETA: 6:06 - loss: 0.2896 - acc: 0.890 - ETA: 6:01 - loss: 0.3260 - acc: 0.857 - ETA: 5:56 - loss: 0.3105 - acc: 0.877 - ETA: 5:51 - loss: 0.3206 - acc: 0.875 - ETA: 4:56 - loss: 0.3724 - acc: 0.812 - ETA: 4:59 - loss: 0.3593 - acc: 0.825 - ETA: 5:00 - loss: 0.3504 - acc: 0.835 - ETA: 5:00 - loss: 0.3438 - acc: 0.842 - ETA: 4:59 - loss: 0.3407 - acc: 0.845 - ETA: 4:57 - loss: 0.3326 - acc: 0.856 - ETA: 4:54 - loss: 0.3264 - acc: 0.861 - ETA: 4:52 - loss: 0.3241 - acc: 0.862 - ETA: 4:49 - loss: 0.3283 - acc: 0.863 - ETA: 4:46 - loss: 0.3271 - acc: 0.865 - ETA: 4:42 - loss: 0.3296 - acc: 0.861 - ETA: 4:38 - loss: 0.3288 - acc: 0.859 - ETA: 4:35 - loss: 0.3248 - acc: 0.862 - ETA: 4:31 - loss: 0.3264 - acc: 0.858 - ETA: 4:27 - loss: 0.3250 - acc: 0.859 - ETA: 4:23 - loss: 0.3244 - acc: 0.860 - ETA: 4:19 - loss: 0.3234 - acc: 0.862 - ETA: 4:15 - loss: 0.3237 - acc: 0.863 - ETA: 4:10 - loss: 0.3225 - acc: 0.864 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.2134 - acc: 0.947 - ETA: 6:06 - loss: 0.2330 - acc: 0.937 - ETA: 6:01 - loss: 0.2738 - acc: 0.906 - ETA: 5:56 - loss: 0.2773 - acc: 0.901 - ETA: 5:52 - loss: 0.2863 - acc: 0.897 - ETA: 5:47 - loss: 0.2999 - acc: 0.888 - ETA: 5:42 - loss: 0.2959 - acc: 0.891 - ETA: 5:37 - loss: 0.3045 - acc: 0.884 - ETA: 5:32 - loss: 0.3083 - acc: 0.880 - ETA: 5:27 - loss: 0.3059 - acc: 0.877 - ETA: 5:22 - loss: 0.3027 - acc: 0.876 - ETA: 5:18 - loss: 0.2991 - acc: 0.881 - ETA: 5:13 - loss: 0.2967 - acc: 0.884 - ETA: 5:08 - loss: 0.2993 - acc: 0.881 - ETA: 5:03 - loss: 0.2982 - acc: 0.881 - ETA: 4:59 - loss: 0.3004 - acc: 0.882 - ETA: 4:54 - loss: 0.2958 - acc: 0.884 - ETA: 4:35 - loss: 0.3127 - acc: 0.868 - ETA: 4:31 - loss: 0.3145 - acc: 0.869 - ETA: 4:27 - loss: 0.3119 - acc: 0.871 - ETA: 4:23 - loss: 0.3100 - acc: 0.874 - ETA: 4:19 - loss: 0.3087 - acc: 0.875 - ETA: 4:15 - loss: 0.3078 - acc: 0.877 - ETA: 4:11 - loss: 0.3061 - acc: 0.877 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.3215 - acc: 0.864 - ETA: 6:06 - loss: 0.3111 - acc: 0.885 - ETA: 6:01 - loss: 0.3149 - acc: 0.871 - ETA: 5:57 - loss: 0.3189 - acc: 0.872 - ETA: 5:52 - loss: 0.3082 - acc: 0.883 - ETA: 5:47 - loss: 0.3182 - acc: 0.869 - ETA: 5:42 - loss: 0.3114 - acc: 0.876 - ETA: 5:37 - loss: 0.3184 - acc: 0.868 - ETA: 5:33 - loss: 0.3203 - acc: 0.863 - ETA: 5:28 - loss: 0.3248 - acc: 0.858 - ETA: 5:23 - loss: 0.3216 - acc: 0.861 - ETA: 5:18 - loss: 0.3210 - acc: 0.862 - ETA: 5:13 - loss: 0.3193 - acc: 0.863 - ETA: 5:08 - loss: 0.3198 - acc: 0.867 - ETA: 5:03 - loss: 0.3182 - acc: 0.870 - ETA: 4:59 - loss: 0.3139 - acc: 0.875 - ETA: 4:54 - loss: 0.3117 - acc: 0.876 - ETA: 4:49 - loss: 0.3119 - acc: 0.877 - ETA: 4:44 - loss: 0.3085 - acc: 0.880 - ETA: 4:39 - loss: 0.3084 - acc: 0.879 - ETA: 4:34 - loss: 0.3094 - acc: 0.878 - ETA: 4:30 - loss: 0.3089 - acc: 0.879 - ETA: 4:25 - loss: 0.3088 - acc: 0.878 - ETA: 4:20 - loss: 0.3113 - acc: 0.876 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.2125 - acc: 0.968 - ETA: 6:06 - loss: 0.2490 - acc: 0.916 - ETA: 6:01 - loss: 0.2669 - acc: 0.892 - ETA: 5:56 - loss: 0.3002 - acc: 0.862 - ETA: 5:51 - loss: 0.3021 - acc: 0.868 - ETA: 5:46 - loss: 0.2896 - acc: 0.878 - ETA: 5:42 - loss: 0.3002 - acc: 0.876 - ETA: 5:37 - loss: 0.2987 - acc: 0.877 - ETA: 5:32 - loss: 0.2945 - acc: 0.885 - ETA: 5:27 - loss: 0.2919 - acc: 0.887 - ETA: 5:22 - loss: 0.2957 - acc: 0.883 - ETA: 5:18 - loss: 0.2902 - acc: 0.887 - ETA: 5:13 - loss: 0.2913 - acc: 0.886 - ETA: 5:08 - loss: 0.2923 - acc: 0.883 - ETA: 5:03 - loss: 0.2971 - acc: 0.879 - ETA: 4:58 - loss: 0.2958 - acc: 0.882 - ETA: 4:54 - loss: 0.2982 - acc: 0.883 - ETA: 4:49 - loss: 0.2972 - acc: 0.884 - ETA: 4:44 - loss: 0.2973 - acc: 0.885 - ETA: 4:39 - loss: 0.2983 - acc: 0.884 - ETA: 4:34 - loss: 0.2980 - acc: 0.885 - ETA: 4:30 - loss: 0.2964 - acc: 0.886 - ETA: 4:25 - loss: 0.2951 - acc: 0.889 - ETA: 4:20 - loss: 0.2968 - acc: 0.886 -

77/78 [============================>.] - ETA: 6:11 - loss: 0.3161 - acc: 0.885 - ETA: 6:06 - loss: 0.3208 - acc: 0.869 - ETA: 6:01 - loss: 0.2989 - acc: 0.888 - ETA: 5:57 - loss: 0.2853 - acc: 0.898 - ETA: 5:52 - loss: 0.2840 - acc: 0.893 - ETA: 5:47 - loss: 0.2908 - acc: 0.895 - ETA: 5:42 - loss: 0.3018 - acc: 0.886 - ETA: 5:37 - loss: 0.2959 - acc: 0.888 - ETA: 5:32 - loss: 0.2973 - acc: 0.886 - ETA: 5:27 - loss: 0.3008 - acc: 0.888 - ETA: 5:23 - loss: 0.2941 - acc: 0.893 - ETA: 5:18 - loss: 0.2974 - acc: 0.892 - ETA: 5:13 - loss: 0.3030 - acc: 0.885 - ETA: 5:08 - loss: 0.3076 - acc: 0.879 - ETA: 5:03 - loss: 0.3105 - acc: 0.876 - ETA: 4:58 - loss: 0.3096 - acc: 0.875 - ETA: 4:54 - loss: 0.3098 - acc: 0.875 - ETA: 4:49 - loss: 0.3077 - acc: 0.876 - ETA: 4:44 - loss: 0.3084 - acc: 0.876 - ETA: 4:39 - loss: 0.3095 - acc: 0.877 - ETA: 4:34 - loss: 0.3081 - acc: 0.879 - ETA: 4:30 - loss: 0.3116 - acc: 0.875 - ETA: 4:25 - loss: 0.3134 - acc: 0.875 - ETA: 4:20 - loss: 0.3107 - acc: 0.875 -

In [116]:
model.save_weights("deep_transfer_3_class_ResNet.h5")

In [117]:
losses, val_losses = history.losses, history.val_losses
fig = plt.figure(figsize=(15, 5))
plt.plot(model.history['loss'], 'g', label="train losses")
plt.plot(model.history['val_loss'], 'r', label="val losses")
plt.grid(True)
plt.title('Training loss vs. Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

TypeError: 'History' object is not subscriptable

In [125]:
from keras.preprocessing import image

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\44_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\44_6_rgb_191.png'
img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\41_10_rgb_191.png'

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\42_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\42_6_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\42_10_rgb_191.png'

img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [122]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:02.623360 [[1. 0. 0.]]


In [124]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.241165 [[1. 0. 0.]]


In [126]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

0:00:00.248045 [[1. 0. 0.]]
